<a href="https://colab.research.google.com/github/yiruchen1993/1st-DL-CVMarathon/blob/feature%2Fdaily_learning/Day021_Transfer_Learning_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [2]:
input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
model = keras.applications.Xception(
            include_top=False,
            weights="imagenet",
            input_tensor=input_tensor,
            input_shape=None,
            pooling=None,
            classes=10)
"""自行填入"""

'''Resnet 50 架構'''
#model=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    #input_tensor=input_tensor,
                                    #pooling=None, classes=10)
model.summary()


83689472/83683744 [==============================] - 1s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
__________________________

## 添加層數

In [3]:
x = model.output

'''可以參考Cifar10實作章節,自行填入'''
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(rate=0.1)(x)

predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))


Model深度： 136


## 鎖定特定幾層不要更新權重

In [4]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

170500096/170498071 [==============================] - 3s 0us/step
(50000, 32, 32, 3)


## Training

In [6]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [7]:
model.fit(x_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
1563/1563 [==============================] - 25s 16ms/step - loss: 1.2665 - accuracy: 0.5644
Epoch 2/10
1563/1563 [==============================] - 25s 16ms/step - loss: 0.8884 - accuracy: 0.6997
Epoch 3/10
1563/1563 [==============================] - 25s 16ms/step - loss: 0.7541 - accuracy: 0.7418
Epoch 4/10
1563/1563 [==============================] - 25s 16ms/step - loss: 0.6361 - accuracy: 0.7845
Epoch 5/10
1563/1563 [==============================] - 25s 16ms/step - loss: 0.5305 - accuracy: 0.8185
Epoch 6/10
1563/1563 [==============================] - 25s 16ms/step - loss: 0.4246 - accuracy: 0.8533
Epoch 7/10
1563/1563 [==============================] - 25s 16ms/step - loss: 0.3283 - accuracy: 0.8878
Epoch 8/10
1563/1563 [==============================] - 25s 16ms/step - loss: 0.2616 - accuracy: 0.9108
Epoch 9/10
1563/1563 [==============================] - 25s 16ms/step - loss: 0.2084 - accuracy: 0.9287
Epoch 10/10
1563/1563 [==============================] - 24s 16m